- get news for each stock
    - scrap links
- get video links associated to article
    - use whisper to get text

# Messari news

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from collections import defaultdict
import datetime
import tqdm
import re

In [2]:
"""def format_news_articles_date(date, scrap_time_dt):
    
    if "minutes" == date.split(" ")[1]:
        hour_shift = int(re.search("(\d+)", date).group(1)) / 60
    elif "hour" == date.split(" ")[1]:
        hour_shift = 1
    elif "hours" == date.split(" ")[1]:
        hour_shift = int(re.search("(\d+)", date).group(1))
    elif "day" == date.split(" ")[1]:
        hour_shift = 24
    elif "days" == date.split(" ")[1]:
        hour_shift = 24 * int(re.search("(\d+)", date).group(1))
        
    return pd.to_datetime(scrap_time_dt - datetime.timedelta(hours=hour_shift))
"""
def format_date(date, scrap_time_dt):
    
    hour_shift = None
    
    if "minutes" == date.split(" ")[1]:
        hour_shift = int(re.search("(\d+)", date).group(1)) / 60
    elif "hour" == date.split(" ")[1]:
        hour_shift = 1
    elif "hours" == date.split(" ")[1]:
        hour_shift = int(re.search("(\d+)", date).group(1))
    elif "day" == date.split(" ")[1]:
        hour_shift = 24
    elif "days" == date.split(" ")[1]:
        hour_shift = 24 * int(re.search("(\d+)", date).group(1))
      
    if hour_shift:
        
        return pd.to_datetime(scrap_time_dt - datetime.timedelta(hours=hour_shift))
    
    else:
        
        date = re.sub("st|nd|th|rd", "", date)
        date = date.split(" ")[0] + " 0" + date.split(" ")[-1] if len(date.split(" ")[-1])==1 else date
        date = pd.to_datetime(date, format="%b %d")
        date = date.replace(year = datetime.datetime.now().year)
        
        return date
    """
    research_articles["Date"] = research_articles.Date.apply(lambda x: re.sub("st|nd|th|rd", "", x))
    research_articles["Date"] = research_articles["Date"].apply(lambda x: x.split(" ")[0] + " 0" + x.split(" ")[-1] if len(x.split(" ")[-1])==1 else x)
    research_articles["Date"] = pd.to_datetime(research_articles.Date, format="%b %d")
    research_articles["Date"] = research_articles['Date'].apply(lambda x: x.replace(year = datetime.datetime.now().year))
    """
        

In [3]:
PATH = "../HEG/FAIRCHAIN/data/wikibase/MESSARI_wikibase.csv"

df_messari = pd.read_csv(PATH)

In [4]:
driver = webdriver.Firefox()

In [5]:
failed_scraping = []

nested_dict = lambda: defaultdict(nested_dict)
news_dict = nested_dict()

for symbol in tqdm.tqdm(df_messari.symbol):

    try:
        scrap_time_dt = datetime.datetime.now()
        scrap_time = scrap_time_dt.strftime("%Y_%m_%d_%H_%M_%S")

        url = f"https://messari.io/asset/{symbol.lower()}"
        driver.get(url)

        time.sleep(10)

        soup = BeautifulSoup(driver.page_source)

        # Get Research article urls
        research_articles = pd.read_html(driver.page_source)[0]
        #research_articles["Article"] = research_articles.Article.apply(lambda x: x.replace("'s", "\\'s"))

        # format research articles date
        #research_articles = format_research_articles_date(research_articles)
        research_articles_timestamps = [format_date(x, scrap_time_dt) for x in research_articles.Date]

        research_articles_urls = []

        for a in soup.find_all("a"):

            if any([x in a.text for x in research_articles.Article]):
                research_articles_urls.append(a["href"])

        research_articles_urls = [(date, "https://messari.io" + x) for date, x in zip(research_articles_timestamps, research_articles_urls)]

        # Get news article urls
        news_articles = pd.read_html(driver.page_source)[1]
        news_articles_urls = []

        # format news articles dates
        news_articles_timestamps = [format_date(x, scrap_time_dt) for x in news_articles.Date]

        for h6 in soup.find_all("h6"):

            if h6.find_previous().name == "a":
                news_articles_urls.append(h6.find_previous()["href"])

        news_articles_urls = [(date, x) for date, x in zip(news_articles_timestamps, news_articles_urls)]

        # store data
        news_dict[symbol][scrap_time]["research_articles_urls"] = research_articles_urls
        news_dict[symbol][scrap_time]["news_articles_urls"] = news_articles_urls
    
    except Exception as e:
        
        print(symbol, "----", e)
        failed_scraping.append(symbol)


  1%|▎                                      | 13/1399 [01:20<2:14:16,  5.81s/it]

DOT ---- No tables found


  2%|▌                                      | 21/1399 [02:07<2:13:21,  5.81s/it]

LINK ---- No tables found


  2%|▌                                      | 22/1399 [02:14<2:23:19,  6.25s/it]

...

POND ---- 'DataFrame' object has no attribute 'Date'


 16%|██████▏                               | 230/1399 [23:01<2:11:03,  6.73s/it]

XYO ---- 'DataFrame' object has no attribute 'Date'


 16%|██████▏                               | 230/1399 [23:04<1:57:16,  6.02s/it]


KeyboardInterrupt: 

In [ ]:
news_dict.keys()

In [ ]:
# convert timestamps to string

# save ton json

# YFINANCE

In [ ]:
import yfinance as yf
import re

In [ ]:
msft = yf.Ticker("MSFT")

In [ ]:
links = [x["link"] for x in msft.news]

In [ ]:
links

In [ ]:
news_articles = []

for url in links:
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    # get affiliate link
    if "Continue reading" in res.text:
        
        affiliate_url = soup.find("a", class_="link caas-button")["href"]
        res = requests.get(affiliate_url)
        
        if "https://www.ft.com/" in affiliate_url:
            
            res = requests.get(affiliate_url)
            soup = BeautifulSoup(res.content)
            script = soup.find_all("script")[0]
            text = re.search('"articleBody":"(.*)","wordCount', script.text).group(1)
            
            news_articles.append(text)
    
    # else base page contains article text --> extract text
    else:
        
        pass
        
    break

# MESSARI

In [ ]:
import requests
import pandas as pd
import collections
import json
import time
import tqdm
import re

### Get all assets

In [ ]:
res = requests.get('https://data.messari.io/api/v2/assets?with-profiles&page=1&limit=1500')
df = pd.DataFrame.from_dict(res.json()["data"])
df.set_index('id', inplace=True)
df.head()

### get news for asset

In [ ]:
news = collections.defaultdict(dict)
symbols_failed = []

for symbol in tqdm.tqdm(df.symbol):
    
    res = requests.get(f'https://data.messari.io/api/v1/news/{symbol}')
    
    try:
        
        news[symbol]["timestamp"] = res.json()["status"]["timestamp"]
        news[symbol]["articles"] = res.json()["data"]
        
    except Exception as e:
        
        print(f"{symbol}: {e}")
        symbols_failed.append(symbol)
        
        if res.json()["status"]["error_message"] != "Not Found":
            
            wait = int(re.search("(\d{2}) seconds", res.json()["status"]["error_message"]).group(1))
        
        else:
            
            continue
        
        time.sleep(wait+5)


In [ ]:
with open('./news.json', 'w') as f:
    json.dump(news, f)

### get all news

In [ ]:
res = requests.get("https://data.messari.io/api/v1/news")

In [ ]:
res.json()["data"]